# Required Libraries


In [1]:
!pip install 'torch==1.6.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
!pip install 'torchvision==0.7.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
!pip install 'pytorch3d==0.2.5'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0 MB 10 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.6.0+cu101 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.6.0+cu101 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.6.0+cu101 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.6.0+cu101 which is incompatible.
Looking in indexes: https://pypi.or

In [2]:
!git clone https://github.com/suraj2906/RobotVisionProject

Cloning into 'RobotVisionProject'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 98 (delta 9), reused 68 (delta 6), pack-reused 0
Unpacking objects: 100% (98/98), done.


##Image configuration

In [5]:
cd /content/RobotVisionProject/sample_images

/content/RobotVisionProject/sample_images


## Loading the image

In [ ]:
from google.colab import files

filename = list(files.upload().keys())[0]

##Setting Paths for Image
- image_path: Path to the input image to be fed to the model
- obj_image: Path to the output object file generated.
- out_img_path: Image result generated by the image.
- Video_path: Path to the 360 view video 

In [6]:
import os

try:
  image_path = '/content/RobotVisionProject/sample_images/%s' % filename
except:
  image_path = '/content/drive/MyDrive/RobotVisionProject/sample_images/test.png'
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]


obj_path = '/content/RobotVisionProject/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/RobotVisionProject/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = '/content/RobotVisionProject/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = '/content/RobotVisionProject/results/pifuhd_final/result_%s_256_display.mp4' % file_name

In [10]:
cd /content

/content


In [11]:
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 120, done.
remote: Total 120 (delta 0), reused 0 (delta 0), pack-reused 120
Receiving objects: 100% (120/120), 222.76 KiB | 12.38 MiB/s, done.
Resolving deltas: 100% (53/53), done.


## Preprocess (for cropping image)

In [12]:
cd /content/lightweight-human-pose-estimation.pytorch/

/content/lightweight-human-pose-estimation.pytorch


In [13]:
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

--2022-11-18 20:19:59--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 23.15.243.166, 2600:1413:b000:38e::4b21, 2600:1413:b000:380::4b21
Connecting to download.01.org (download.01.org)|23.15.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87959810 (84M) [application/octet-stream]
Saving to: ‘checkpoint_iter_370000.pth’

checkpoint_iter_370 100%[===================>]  83.88M   333MB/s    in 0.3s    

2022-11-18 20:19:59 (333 MB/s) - ‘checkpoint_iter_370000.pth’ saved [87959810/87959810]



##Isolating the Sculpture

### Processing Involved:
1. Blob detection and bounding box around the image.
2. Cropping the image to the region of interest.
3. Passing the cropped image to:
  - Pose detection layer
  - Texture inference and keypoints identification layer
4. Passing the combined generated output through Pose Entry layer and to analyse full output of the image.

In [14]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [ ]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

get_rect(net.cuda(), [image_path], 512)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


## Download the Trained Model

In [4]:
cd /content/RobotVisionProject/

/content/RobotVisionProject


In [ ]:
!sh ./scripts/download_trained_model.sh

## Run PIFuHD!


In [ ]:
# Warning: all images with the corresponding rectangle files under -i will be processed. 
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

# seems that 256 is the maximum resolution that can fit into Google Colab. 
# If you want to reconstruct a higher-resolution mesh, please try with your own machine. 

Resuming from  ./checkpoints/pifuhd.pt
test data size:  5
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/5 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_AmbedkarSt_256.obj
/content/drive/MyDrive/pifuhd/lib/mesh_util.py:77: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  verts, faces, normals, values = measure.marching_cubes_lewiner(sdf, thresh)
 20% 1/5 [00:06<00:25,  6.27s/it]./results/pifuhd_final/recon/result_USCapitol_256.obj
 40% 2/5 [00:11<00:17,  5.81s/it]./results/pifuhd_final/recon/result_human-statue-500x500_256.obj
 60% 3/5 [00:18<00:12,  6.45s/it]./results/pifuhd_final/recon/result_liberty_256.obj
 80% 4/5 [00:24<00:06,  6.19s/it]./results/pifuhd_final/recon/result_unity_256.obj
100% 5/5 [00:30<00:00,  6.16s/it]


## Render the result

In [ ]:
from lib.colab_util import generate_video_from_obj, set_renderer, video

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)

## Statue of Mr. B. R. Ambedkar

In [5]:
from lib.colab_util import generate_video_from_obj, set_renderer, video

obj_path = "/content/RobotVisionProject/results/pifuhd_final/recon/result_AmbedkarSt_256.obj"
out_img_path = "/content/RobotVisionProject/results/pifuhd_final/recon/result_AmbedkarSt_256.png"
video_path = '/content/RobotVisionProject/results/pifuhd_final/recon/result_AmbedkarSt_256.mp4'
video_display_path = '/content/RobotVisionProject/results/pifuhd_final/result_AmbedkarSt_256_display.mp4'
renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)

  0%|          | 0/90 [00:00<?, ?it/s]

##Statue of American Civil Rights Activist Daisy Bates

In [6]:
from lib.colab_util import generate_video_from_obj, set_renderer, video

obj_path = "/content/RobotVisionProject/results/pifuhd_final/recon/result_USCapitol_256.obj"
out_img_path = "/content/RobotVisionProject/results/pifuhd_final/recon/result_USCapitol_256.png"
video_path = '/content/RobotVisionProject/results/pifuhd_final/recon/result_USCapitol_256.mp4'
video_display_path = '/content/RobotVisionProject/results/pifuhd_final/result_USCapitol_256_display.mp4'
renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)

  0%|          | 0/90 [00:00<?, ?it/s]

##Victorian Human Statue


In [7]:
obj_path = "/content/RobotVisionProject/results/pifuhd_final/recon/result_human-statue-500x500_256.obj"
out_img_path = "/content/RobotVisionProject/results/pifuhd_final/recon/result_human-statue-500x500_256.png"
video_path = '/content/RobotVisionProject/results/pifuhd_final/recon/result_human-statue-500x500_256.mp4'
video_display_path = '/content/RobotVisionProject/results/pifuhd_final/result_human-statue-500x500_256_display.mp4'
renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)

  0%|          | 0/90 [00:00<?, ?it/s]